In [7]:
import numpy as np
import pandas as pd
import os
from numpy.linalg import norm
import json


In [2]:
with open('./input_answers.txt') as fin:
    inp = fin.read()
    print(inp)

This is the first sentence.This is a nice day.Good day.I am not sure why.
This is the second sentence.This is not the first sentence. We know that.
This is the third sentence.This is not the first sentence.This is not the first sentence.
This is not the third sentence.We know that.
The sky is blue this morning. It was not like this yesterday.


In [ ]:
# biomedbert code extract embeddings "./input_answers.txt" 
# "gs://ekaba-assets/bert_model_sat_18th_april/biomedbert-8M.txt" 
# "gs://ekaba-assets/bert_model_sat_18th_april/bert_config.json" 
# "gs://ekaba-assets/bert_model_sat_18th_april/model.ckpt-1000000"


In [13]:
path = 'outputinput_answers.jsonl'

with open('outputinput_answers.jsonl') as fin:
    output_jsonl = fin.read()

In [75]:
def get_sent_embed(output_jsonl) :
    
    #We will run the model and get the outputs
    json_lines = output_jsonl.split('\n')
    
    #Removing the blank strings
    json_lines =  list(filter(None,json_lines))
    
    #getting the dimensions & getting the output of the query
    line_q = json.loads(json_lines[0])
    embed_size = len(line_q['features'][0]['layers'][0]['values'])
    
    #Temp list for saving the tokens
    token_temp_q = []
    
    #array for saving the embeddings
    feat_embed_q = np.zeros((len(line_q['features']),embed_size))
    
    #Getting the final df
    df_query = pd.DataFrame()
    
    for j,feature in enumerate(line_q['features']):
        
        if feature['token'] != '[UNK]':
            token_temp_q.append(feature['token'])
            feat_embed_q[j] = feature['layers'][0]['values']


    #mean_embed
    avg_embed_q =  np.mean(feat_embed_q[1:len(feat_embed_q)-1],axis=0)
    
    #final_output_embeddings
    tokens_query = ' '.join(token_temp_q[1:len(token_temp_q)-1])

    #final query dataframe
    df_query['documents'] = [tokens_query]
    df_query['embedding'] = [avg_embed_q]
    
    
    #--------------------------------------- answers ----------------------------------------------#
    
    #Defining the lists
    sent_embed = []
    tokens = []
    
    #Getting the final df
    df_ans = pd.DataFrame()
    
    #Running for the sentence
    for i in range(1,len(json_lines)):
        line = json.loads(json_lines[i])
        
        #array for saving the embeddings
        feat_embed = np.zeros((len(line['features']),embed_size))
        
        #Temp list for saving the tokens
        token_temp = []
        for j,feature in enumerate(line['features']):
            
            if feature['token'] != '[UNK]':
                token_temp.append(feature['token'])
                feat_embed[j] = feature['layers'][0]['values']
            

        #sanity checks
        avg_embed =  np.mean(feat_embed[1:len(feat_embed)-1],axis=0)

        
        #final_output_embeddings
        sent_embed.append(avg_embed)
        tokens.append(' '.join(token_temp[1:len(token_temp)-1]))
        
         
        
    df_ans['documents'] = tokens
    df_ans['embedding'] = sent_embed
    
    return df_query ,df_ans
        

In [76]:
def get_cosine_sim(output_jsonl) :
    
    #We will run the model and get the outputs
    
    #Getting the dataframes
    df_query ,df_ans = get_sent_embed(output_jsonl)
    
    #Query embedding
    query_embed = df_query['embedding'].values[0]
    query_embed_norm = query_embed/norm(query_embed)
    
    #Answers embedding
    list_embed = df_ans['embedding'].tolist()
    
    #getting the answer embedding
    ans_embed =  np.stack(list_embed, axis=0)
    ans_embed_norm = ans_embed/norm(ans_embed,axis=1,keepdims= True)
    
    #find the cosine similarity
    cos_sim = np.dot(ans_embed_norm,query_embed_norm)
    
    return cos_sim.argsort()[::-1],np.sort(cos_sim)[::-1]

In [77]:
df_query ,df_ans = get_sent_embed(output_jsonl)

In [78]:
ind,scores = get_cosine_sim(output_jsonl)

In [79]:
ind

array([0, 3, 2, 1])

In [80]:
scores

array([0.80510926, 0.76211442, 0.74950555, 0.7376297 ])

In [81]:
df_query

,documents,embedding
0,this is the first sentence this is a nice day ...,"[0.07916590476190477, -0.2536499523809524, 0.1..."


In [83]:
df_ans

,documents,embedding
0,this is the second sentence this is not the fi...,"[0.09111094117647064, -0.372554, 0.01575058823..."
1,this is the third sentence this is not the fir...,"[0.15030149999999995, -0.11581355000000002, -0..."
2,this is not the third sentence we know that,"[-0.2963907272727273, -0.4580452727272728, -0...."
3,the sky is blue this morning it was not like t...,"[0.530984625, -0.535006125, 0.540343375, -0.45..."
